# PFDA Project

### by E. Qejvani
***

## Part 1 - Preparing the dataset.
***

* Uploading the libraries.
* Keeping the information we are interested in.
* Preparing the dataset for analyzing.

In [1]:
# Importing the libraries.
import pandas as pd 
# Importing project_functions file where I'm storing all the functions to use in this project. 
from project_functions import *         

In [2]:
# Uploading the dataset to my data frame, skipping the first 23 rows as the dataset starts at row 24. 
# Setting low_memory=False - easier to upload in big datasets.
df = pd.read_csv('./data/data_valentia.csv', skiprows=23, low_memory=False)

# Creating a subset using only the data I need for this project.
df_valentia = df[['date', 'rain', 'temp', 'wetb', 'dewpt', 'rhum', 'vappr', 'msl', 'wdsp', 'wddir']].copy()

# df_valentia = df.drop(columns=['ww', 'sun', 'w', 'vis', 'clht', 'clamt', 'ind', 'ind.1', 'ind.2', 'ind.3', 'ind.4']) 

df_valentia.head(5)

,date,rain,temp,wetb,dewpt,rhum,vappr,msl,wdsp,wddir
0,01-jan-1944 00:00,0.0,9.3,8.3,7.2,86,10.1,1034.3,12,300
1,01-jan-1944 01:00,0.0,8.9,8.2,7.2,90,10.3,1033.9,9,290
2,01-jan-1944 02:00,0.0,9.4,8.1,6.6,83,9.7,1033.4,11,280
3,01-jan-1944 03:00,0.0,9.3,8.3,7.2,86,10.2,1032.8,11,280
4,01-jan-1944 04:00,0.5,8.6,7.8,7.2,89,9.9,1032.4,11,300


### Checking the dataset.

In [3]:
# Information about our dataset.
df_valentia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709320 entries, 0 to 709319
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    709320 non-null  object
 1   rain    709320 non-null  object
 2   temp    709320 non-null  object
 3   wetb    709320 non-null  object
 4   dewpt   709320 non-null  object
 5   rhum    709320 non-null  object
 6   vappr   709320 non-null  object
 7   msl     709320 non-null  object
 8   wdsp    709320 non-null  object
 9   wddir   709320 non-null  object
dtypes: object(10)
memory usage: 54.1+ MB


### Changing the data types in my dataset.

As we can see from the above all data types in the dataset are object type. To change this I am going to:
- Change the date column type to date type data.
- Change the rest of the columns to float type data.
- For both changes I'm creating a function in the `project_functions.py` file.

In [4]:
# 
columns_to_convert = ['rain', 'temp', 'wetb', 'dewpt', 'rhum', 'vappr', 'msl', 'wdsp', 'wddir']
df_valentia = convert_columns_to_float(df_valentia, columns_to_convert)

In [5]:
df_valentia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709320 entries, 0 to 709319
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    709320 non-null  object 
 1   rain    709278 non-null  float64
 2   temp    709260 non-null  float64
 3   wetb    709260 non-null  float64
 4   dewpt   709260 non-null  float64
 5   rhum    709260 non-null  float64
 6   vappr   709260 non-null  float64
 7   msl     709260 non-null  float64
 8   wdsp    709275 non-null  float64
 9   wddir   709211 non-null  float64
dtypes: float64(9), object(1)
memory usage: 54.1+ MB


### Checking if any data is missing - NAN or null values.

In [6]:
# Checking for missing data.
# df_valentia.isnull().sum()
# df_valentia.isna().sum()
# (df_valentia == ' ').sum()

### Checking for single space values `' '`

In Assignment Six of this module, I found out that using `.isna()` and `.isnull()` alone is insufficient for cleaning the dataset. Sometimes, cells can contain an empty value represented by a single space (stored as a string). Therefore, in the next section, I will check for any cells that contain a single space as their value.

### Counting the rows that hold empty cells.

* I created a function in the `project_functions` file that counts all the rows that hold empty cells. 

In [7]:
# calling the funtion from the 'project_functions.py' file.
count_rows_with_empty_or_space_cells_detail(df_valentia)

Rows with 0 empty cells: 709144
Rows with 1 empty cells: 75
Rows with 2 empty cells: 41
Rows with 6 empty cells: 22
Rows with 7 empty cells: 37
Rows with 8 empty cells: 1
Total rows with at least one empty values: 176


176

### Cleaning the rows with empty cells.

* Because of the big size of the dataset (709320 records) I decided to remove all the rows that contain empty cells.
* The total records after removing 176 rows becomes 709144.

In [8]:
remove_rows_with_missing_data(df_valentia)
df_valentia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709320 entries, 0 to 709319
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    709320 non-null  object 
 1   rain    709278 non-null  float64
 2   temp    709260 non-null  float64
 3   wetb    709260 non-null  float64
 4   dewpt   709260 non-null  float64
 5   rhum    709260 non-null  float64
 6   vappr   709260 non-null  float64
 7   msl     709260 non-null  float64
 8   wdsp    709275 non-null  float64
 9   wddir   709211 non-null  float64
dtypes: float64(9), object(1)
memory usage: 54.1+ MB


Changing the `date` column to date 

### Working with the data types.